In [3]:
!pip install keras tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 4.0 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 5.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 4.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 6.1 MB/s 

In [ ]:
import os
import re
import json
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import pyarrow
import time


In [2]:
os.chdir("./")
print("Current directory:", os.getcwd())

import pandas as pd
import glob
import os
# Find all matching files
files = sorted(glob.glob("sequence_hdfs_*.bin"), key=os.path.getmtime)
if not files:
    raise FileNotFoundError("No sequence_hdfs_*.bin files found!")
# Pick the most recent one
latest_file = files[-1]
print(f"Loading: {latest_file}")
# Load as Feather
df_sequence = pd.read_feather(latest_file)
print(df_sequence.columns)
print(df_sequence.shape)
print(df_sequence.head(5))

Current directory: /home/andy/6309GP
Loading: sequence_hdfs_20251024_183457.bin
Index(['SessionId', 'EventSequence', 'Label'], dtype='object')
(575061, 3)
                  SessionId  \
0  blk_-1608999687919862906   
1   blk_7503483334202473044   
2  blk_-3544583377289625738   
3  blk_-9073992586687739851   
4   blk_7854771516489510256   

                                       EventSequence  Label  
0  E5 E22 E5 E5 E11 E11 E9 E9 E11 E9 E26 E26 E26 ...      0  
1  E5 E5 E22 E5 E11 E9 E11 E9 E11 E9 E26 E26 E26 ...      0  
2  E5 E22 E5 E5 E11 E9 E11 E9 E11 E9 E3 E26 E26 E...      1  
3  E5 E22 E5 E5 E11 E9 E11 E9 E11 E9 E26 E26 E26 ...      0  
4  E5 E5 E22 E5 E11 E9 E11 E9 E11 E9 E26 E26 E26 ...      0  


In [3]:
from sklearn.model_selection import train_test_split

def split_sequence_dataset(df, test_size=0.3, random_state=42):
    """
    Split a sequence dataset into training and evaluation sets.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with columns ['SessionId', 'EventSequence', 'Label'].
    test_size : float
        Fraction of the dataset to reserve for evaluation (default 0.3).
    random_state : int
        Random seed for reproducibility.

    Returns
    -------
    sequence_train : pd.DataFrame
    sequence_eval : pd.DataFrame
    """
    sequence_train, sequence_eval = train_test_split(
        df,
        test_size=test_size,
        random_state=random_state,
        stratify=df["Label"],  # preserve label distribution
        shuffle=True
    )
    return sequence_train.reset_index(drop=True), sequence_eval.reset_index(drop=True)

sequence_train, sequence_eval = split_sequence_dataset(df_sequence)
print(sequence_train.shape, sequence_eval.shape)
print(sequence_train["Label"].value_counts(normalize=True))
print(sequence_eval["Label"].value_counts(normalize=True))

(402542, 3) (172519, 3)
Label
0    0.970719
1    0.029281
Name: proportion, dtype: float64
Label
0    0.970722
1    0.029278
Name: proportion, dtype: float64


In [4]:
def get_normal_subset(df_train, label_col="Label"):
    return df_train[df_train[label_col] == 0].reset_index(drop=True)

sequence_train_normal = get_normal_subset(sequence_train)
print(sequence_train_normal.shape)
print(sequence_train_normal["Label"].value_counts())

(390755, 3)
Label
0    390755
Name: count, dtype: int64


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

# Identity tokenizer function
def identity_tokenizer(s: str):
    # For pre-tokenized input (space-separated)
    return s.split()

# Vectorizer setup
tfidf_id = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False, preprocessor=None)
tfidf_matrix_id = tfidf_id.fit_transform(df_sequence["EventSequence"].astype(str))

# Get feature names
feature_names_id = np.array(tfidf_id.get_feature_names_out())

# Prepare X and y
X = tfidf_matrix_id
y = df_sequence["Label"].values

# Split into train and eval sets
X_train, X_eval, y_train, y_eval = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Filter for normal class (0) only in training set
mask_normal = (y_train == 0)
X_train_normal = X_train[mask_normal]
y_train_normal = y_train[mask_normal]  # Make sure to filter y_train as well

print("Train normal:", X_train_normal.shape, "Eval:", X_eval.shape)
print("Labels (train normal):", np.unique(y_train_normal))


/home/andy/andyVenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Train normal: (390755, 29) Eval: (172519, 29)
Labels (train normal): [0]


threat to validity: the use of autoencoder expects a matrix with the same size. This is a poor combination when applied with MCV and TF-IDF: 

IF there are novel templates in eval set in comparison to training set, the autoencoder breaks. 

In post, it would be better to use an embedding such as word to vect where the matrix is of a fixed dimension. 

In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

def evaluate_mlp_results(result_df):
    """
    Compute classification metrics from MLP results.

    Parameters
    ----------
    result_df : pd.DataFrame
        Must contain columns: ['true_label', 'pred_label']

    Returns
    -------
    metrics : dict
        Dictionary of precision, recall, f1, and auc scores.
    """
    
    y_true = result_df["true_label"].astype(int)
    y_pred = result_df["pred_label"].astype(int)

    # Compute classification metrics
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    accuracy = accuracy_score(y_true, y_pred)

    # ROC AUC can be computed if the labels are binary (0 or 1)
    try:
        auc = roc_auc_score(y_true, y_pred)
    except ValueError:
        auc = None  # If AUC can't be computed, set it to None

    # Prepare metrics dictionary
    metrics = {
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "auc": auc
    }

    print("\n📊 Evaluation metrics:")
    for k, v in metrics.items():
        if v is not None:
            print(f"{k:10s}: {v:.4f}")
        else:
            print(f"{k:10s}: Not available (AUC could not be computed)")

    return metrics


In [10]:
def train_and_evaluate_mlp(X_train, X_eval, y_train, y_eval, hidden_layer_sizes=(128, 64), epochs=30, batch_size=32, lr=1e-3):
    """
    Train an MLPClassifier and evaluate its performance on an evaluation set.
    
    Parameters
    ----------
    X_train : array-like
        Training data.
    X_eval : array-like
        Evaluation data.
    y_train : array-like
        Labels for training data.
    y_eval : array-like
        Labels for evaluation data.
    hidden_layer_sizes : tuple
        Sizes of the hidden layers.
    epochs : int
        Number of epochs for training.
    batch_size : int
        Batch size for training.
    lr : float
        Learning rate.

    Returns
    -------
    result_df : pd.DataFrame
        DataFrame with columns ['true_label', 'pred_label', 'reconstruction_error'].
    model : MLPClassifier
        Trained model.
    """

    from sklearn.neural_network import MLPClassifier
    import pandas as pd
    import numpy as np

    print(f"Training MLP on {X_train.shape[0]} samples, evaluating on {X_eval.shape[0]} samples...")

    # Initialize the MLPClassifier
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                          max_iter=epochs,
                          learning_rate_init=lr,
                          batch_size=batch_size,
                          verbose=True)

    # Train the model
    model.fit(X_train, y_train)

    # Predict on the evaluation set
    y_pred = model.predict(X_eval)

    # Build output DataFrame
    result_df = pd.DataFrame({
        "true_label": y_eval,
        "pred_label": y_pred
    })

    print("\n✅ Training complete.")
    print(result_df["pred_label"].value_counts())

    return result_df, model


In [8]:
import pandas as pd
import numpy as np

def analyze_classification_results_by_label(result_df):
    """
    Compute summary statistics (mean, std, count) of predictions by label 
    for normal (label=0) and anomaly (label=1) samples.

    Parameters
    ----------
    result_df : pd.DataFrame
        DataFrame with at least columns ['true_label', 'pred_label'].

    Returns
    -------
    summary_df : pd.DataFrame
        Summary of prediction count, true positive, false positive, etc. by label.
    """

    if not {"true_label", "pred_label"}.issubset(result_df.columns):
        raise ValueError("result_df must contain 'true_label' and 'pred_label' columns")

    # Create a confusion matrix
    confusion_matrix = pd.crosstab(result_df["true_label"], result_df["pred_label"], 
                                   rownames=["True Label"], colnames=["Predicted Label"])

    # Calculate summary statistics for each label (Normal and Anomaly)
    summary = (
        result_df.groupby("true_label")["pred_label"]
        .agg(["count", "mean", "std"])
        .rename(index={0: "Normal (0)", 1: "Anomaly (1)"})
    )

    print("\n📊 Classification Result Summary by Label:")
    print(summary)

    # Additional interpretation based on counts and prediction distribution
    normal_pred_count = summary.loc["Normal (0)", "count"]
    anomaly_pred_count = summary.loc["Anomaly (1)", "count"]

    print(f"\nNormal predictions count: {normal_pred_count}")
    print(f"Anomaly predictions count: {anomaly_pred_count}")

    # Confusion matrix interpretation
    print("\nConfusion Matrix:")
    print(confusion_matrix)

    # Add insights on model's performance for Normal vs. Anomaly detection
    true_positives = confusion_matrix.loc[1, 1] if 1 in confusion_matrix.index and 1 in confusion_matrix.columns else 0
    false_positives = confusion_matrix.loc[0, 1] if 0 in confusion_matrix.index and 1 in confusion_matrix.columns else 0
    false_negatives = confusion_matrix.loc[1, 0] if 1 in confusion_matrix.index and 0 in confusion_matrix.columns else 0
    true_negatives = confusion_matrix.loc[0, 0] if 0 in confusion_matrix.index and 0 in confusion_matrix.columns else 0

    # Calculate precision, recall, and F1 for both classes
    precision_normal = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0
    recall_normal = true_negatives / (true_negatives + false_negatives) if (true_negatives + false_negatives) > 0 else 0
    precision_anomaly = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    recall_anomaly = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

    print(f"\nPrecision (Normal): {precision_normal:.4f}")
    print(f"Recall (Normal): {recall_normal:.4f}")
    print(f"Precision (Anomaly): {precision_anomaly:.4f}")
    print(f"Recall (Anomaly): {recall_anomaly:.4f}")

    return summary


In [22]:
# Assuming you have labeled data
result_df, mlp_model = train_and_evaluate_mlp(
    X_train, X_eval, y_train=y_train, y_eval=y_eval,
    hidden_layer_sizes=(128, 64), epochs=10, batch_size=32, lr=1e-3
)

print(result_df.head(10))
metrics = evaluate_mlp_results(result_df)
analyze_classification_results_by_label(result_df)


Training MLP on 402542 samples, evaluating on 172519 samples...
Iteration 1, loss = 0.00473507
Iteration 2, loss = 0.00191899
Iteration 3, loss = 0.00168320
Iteration 4, loss = 0.00158739
Iteration 5, loss = 0.00151933
Iteration 6, loss = 0.00144901
Iteration 7, loss = 0.00142690
Iteration 8, loss = 0.00140413
Iteration 9, loss = 0.00136318
Iteration 10, loss = 0.00131422


/home/andy/andyVenv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(



✅ Training complete.
pred_label
0    167466
1      5053
Name: count, dtype: int64
   true_label  pred_label
0           0           0
1           0           0
2           0           0
3           0           0
4           0           0
5           0           0
6           0           0
7           0           0
8           0           0
9           0           0

📊 Evaluation metrics:
precision : 0.9954
recall    : 0.9958
f1_score  : 0.9956
accuracy  : 0.9997
auc       : 0.9979

📊 Classification Result Summary by Label:
              count      mean       std
true_label                             
Normal (0)   167468  0.000137  0.011718
Anomaly (1)    5051  0.995842  0.064352

Normal predictions count: 167468
Anomaly predictions count: 5051

Confusion Matrix:
Predicted Label       0     1
True Label                   
0                167445    23
1                    21  5030

Precision (Normal): 0.9999
Recall (Normal): 0.9999
Precision (Anomaly): 0.9958
Recall (Anomaly): 0.9954


,count,mean,std
true_label,,,
Normal (0),167468,0.000137,0.011718
Anomaly (1),5051,0.995842,0.064352


In [ ]:
metrics = evaluate_mlp_results(result_df)


📊 Evaluation metrics:
precision : 0.9529
recall    : 0.6575
f1_score  : 0.7781
auc       : 0.9989


In [23]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

def train_and_evaluate_rf(X_train, X_eval, y_train, y_eval,
                          n_estimators=100, max_depth=None, contamination=0.05):
    """
    Train a Random Forest Classifier on X_train and evaluate on X_eval.

    Parameters
    ----------
    X_train : array-like or sparse matrix
        Training feature matrix (labeled samples).
    X_eval : array-like or sparse matrix
        Evaluation feature matrix.
    y_train, y_eval : array-like
        Labels for training and evaluation sets.
    n_estimators : int
        Number of trees in the forest.
    max_depth : int, optional
        The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure.
    contamination : float
        Expected fraction of anomalies in the data (for anomaly detection). Not used directly with RF, 
        but can be used for thresholding when evaluating predictions.

    Returns
    -------
    result_df : pd.DataFrame
        DataFrame with columns:
        ['true_label', 'pred_label', 'probability']
    model : RandomForestClassifier
        Trained Random Forest model.
    """

    print(f"Training Random Forest on {X_train.shape[0]} samples, "
          f"evaluating on {X_eval.shape[0]} samples...")

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, verbose=1)
    model.fit(X_train, y_train)

    # Predict on the evaluation set
    y_pred = model.predict(X_eval)  # Predicted labels (0 for normal, 1 for anomaly)
    probabilities = model.predict_proba(X_eval)[:, 1]  # Probability for the anomaly class (class 1)

    # Build result DataFrame
    result_df = pd.DataFrame({
        "true_label": y_eval,
        "pred_label": y_pred,
        "probability": probabilities
    })

    # Display summary
    print("\n✅ Training complete.")
    print("Predicted label counts:")
    print(result_df["pred_label"].value_counts())
    print("Mean anomaly probability:", np.mean(probabilities))

    return result_df, model


In [24]:
# Train and evaluate
result_df, rf_model = train_and_evaluate_rf(
    X_train=X_train,
    X_eval=X_eval,
    y_train=y_train,
    y_eval=y_eval,
    n_estimators=100,      # You can adjust the number of trees
    max_depth=10,          # You can set the max depth of trees
    contamination=0.05     # This can be used for thresholding, not directly in RF
)

# Optional: evaluate performance metrics
from sklearn.metrics import classification_report, roc_auc_score

# Classification report
print(classification_report(result_df["true_label"], result_df["pred_label"]))

# ROC AUC Score
print("ROC AUC:", roc_auc_score(result_df["true_label"], result_df["probability"]))


Training Random Forest on 402542 samples, evaluating on 172519 samples...


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    4.3s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    9.0s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s



✅ Training complete.
Predicted label counts:
pred_label
0    167479
1      5040
Name: count, dtype: int64
Mean anomaly probability: 0.029180942515229832
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    167468
           1       1.00      1.00      1.00      5051

    accuracy                           1.00    172519
   macro avg       1.00      1.00      1.00    172519
weighted avg       1.00      1.00      1.00    172519

ROC AUC: 0.9999976811155398


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
